In [122]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import numpy as np
import jieba as jb
from pandas import *
import re

#### 加载数据集用DataFrame存储

In [123]:
labels = [];datas = [];
fr = open('toutiao_cat_data.txt',encoding='utf-8')
for line in fr.readlines():
    lineArr = line.strip().split("_!_")#strip()删除换行符，用split("_!_")把每行文本按照_!_分开
    labels.append(lineArr[2])#标签
    datas.append(lineArr[3])#特征：标题+新闻关键词+ lineArr[4]
#用DataFrame()存储
df = pandas.DataFrame()
df['data'] = datas
df['label'] = labels

In [124]:
df

,data,label
0,京城最值得你来场文化之旅的博物馆,news_culture
1,发酵床的垫料种类有哪些？哪种更好？,news_culture
2,上联：黄山黄河黄皮肤黄土高原。怎么对下联？,news_culture
3,林徽因什么理由拒绝了徐志摩而选择梁思成为终身伴侣？,news_culture
4,黄杨木是什么树？,news_culture
...,...,...
382683,A10处理器iPhone SE二代值得期待吗？,news_tech
382684,先进战机叛逃将带来重大损失，美军如何防止F22飞行员驾机叛逃？,news_military
382685,又一国领导人放话，只要普京下令，数万大军“碾压”美国白宫！,news_world
382686,如何看待美国总统连续撕毁美国签署的国际协议的举动？,news_world


In [125]:
#列表中添加类型id
df['id'] = df['label'].factorize()[0]#factorize创建数字表示类别变量，对每一个类别映射一个id
id_df = df[['label', 'id']].drop_duplicates().sort_values('id').reset_index(drop=True)#删除重复项、根据id排序、重置index

In [126]:
df

,data,label,id
0,京城最值得你来场文化之旅的博物馆,news_culture,0
1,发酵床的垫料种类有哪些？哪种更好？,news_culture,0
2,上联：黄山黄河黄皮肤黄土高原。怎么对下联？,news_culture,0
3,林徽因什么理由拒绝了徐志摩而选择梁思成为终身伴侣？,news_culture,0
4,黄杨木是什么树？,news_culture,0
...,...,...,...
382683,A10处理器iPhone SE二代值得期待吗？,news_tech,7
382684,先进战机叛逃将带来重大损失，美军如何防止F22飞行员驾机叛逃？,news_military,8
382685,又一国领导人放话，只要普京下令，数万大军“碾压”美国白宫！,news_world,10
382686,如何看待美国总统连续撕毁美国签署的国际协议的举动？,news_world,10


In [127]:
#查看14种类别的数量
for name, group in df.groupby(df.columns[1]):
    print(name,len(group))

news_agriculture 19322
news_car 35785
news_culture 28031
news_edu 27058
news_entertainment 39396
news_finance 27085
news_game 29300
news_house 17672
news_military 24984
news_sports 37568
news_story 6273
news_tech 41543
news_travel 21422
news_world 26909
stock 340


#### 处理文本：过滤标点、停用词

In [128]:
#删除标点函数
def remove_punctuation(line):
    line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    line = rule.sub('',line)
    return line
#加载停用词
def stopwordslist():  
    stopwords = [line.strip() for line in open("chineseStopWords.txt", 'r', encoding='utf-8').readlines()]  
    return stopwords  
stopwords = stopwordslist()

In [129]:
#删除标点
df['data'] = df['data'].apply(remove_punctuation)#删除标点
#分词、过滤停用词并写入cutWords_list
cutWords_list = []
content_series = df['data']

for i in range(len(content_series)):
    content = content_series.iloc[i]
    cutWords = [k for k in jb.cut(content, True) if k not in stopwords]
    cutWords_list.append(cutWords)

In [130]:
cutWords_list[0:5]

[['京城', '值得', '场', '文化', '旅', '博物', '博物馆'],
 ['发酵', '床', '垫料', '种类', '哪种', '更好'],
 ['上联', '黄山', '黄河', '黄皮', '黄皮肤', '皮肤', '黄土', '黄土高原', '高原', '下联'],
 ['林徽因', '理由', '拒绝', '徐志摩', '志摩', '选择', '梁思成', '成为', '终身', '终身伴侣', '伴侣'],
 ['黄杨', '黄杨木', '杨木', '树']]

In [131]:
df

,data,label,id
0,京城最值得你来场文化之旅的博物馆,news_culture,0
1,发酵床的垫料种类有哪些哪种更好,news_culture,0
2,上联黄山黄河黄皮肤黄土高原怎么对下联,news_culture,0
3,林徽因什么理由拒绝了徐志摩而选择梁思成为终身伴侣,news_culture,0
4,黄杨木是什么树,news_culture,0
...,...,...,...
382683,A10处理器iPhoneSE二代值得期待吗,news_tech,7
382684,先进战机叛逃将带来重大损失美军如何防止F22飞行员驾机叛逃,news_military,8
382685,又一国领导人放话只要普京下令数万大军碾压美国白宫,news_world,10
382686,如何看待美国总统连续撕毁美国签署的国际协议的举动,news_world,10


#### word2vec模型

In [132]:
import warnings
warnings.filterwarnings('ignore')

In [133]:
#word2vec模型实例化对象
from gensim.models import Word2Vec
#输出向量位维数size=100，迭代次数iter=10，丢掉词频少于min_count的词语
word2vec_model = Word2Vec(cutWords_list, size=200, iter=10, min_count=0)#将cutWords_list中的的词映射到语义空间中

In [134]:
#通过word2vec对象的most_similar方法获取词义相近的词
word2vec_model.wv.most_similar("特朗普")

[('后任', 0.6479679346084595),
 ('白宫', 0.6259499788284302),
 ('新官上任', 0.6157052516937256),
 ('奥巴马', 0.6074644923210144),
 ('总统', 0.6067150831222534),
 ('英法德', 0.6033245921134949),
 ('孤行', 0.6003737449645996),
 ('政客', 0.6002194881439209),
 ('巴马', 0.5974191427230835),
 ('奥巴', 0.5955947637557983)]

In [135]:
word2vec_model.similarity('奔驰','奥迪')

0.78269786

#### 词向量表示

In [136]:
#将每条新闻标题向量化表示
'''
对于一条新闻标题，首先获取这条标题的每一个分词在word2vec模型的相关性向量，然后将这些相关性向量求和取平均数，
结果即为这条新闻标题在word2vec模型中的相关性向量。
实例化Word2Vec对象时，关键字参数size定义为100，则相关性矩阵都为100维。
getVector函数获取每个标题的词向量，传入2个参数，第1个参数是文章分词的结果，第2个参数是word2vec模型对象。
'''
def get_contentVector(cutWords, word2vec_model):
    vector_list = [word2vec_model.wv[k] for k in cutWords if k in word2vec_model]
    contentVector = np.array(vector_list).mean(axis=0)
    return contentVector

#循环处理每一条新闻标题
contentVector_list = []
for i in range(len(cutWords_list)):
    cutWords = cutWords_list[i]
    #contentVector_list.append(get_contentVector(cutWords, word2vec_model))
    cVect = get_contentVector(cutWords, word2vec_model)
    
#当min_count过大时一些词被丢掉导致维度为nan,需要过滤维度不正确的值
#     if(cVect.shape!=(200,)):
#          cVect = np.zeros(200,)
        
    contentVector_list.append(cVect)
    
#将列表转为特征矩阵   
X = np.array(contentVector_list) #21662 

In [137]:
X.shape

(382688, 200)

In [138]:
#对新闻类别做标签编码
from sklearn.preprocessing import LabelEncoder

labelEncoder = LabelEncoder()
y = labelEncoder.fit_transform(df['id'])

In [139]:
y

array([ 0,  0,  0, ..., 10, 10, 11], dtype=int64)

In [140]:
y.shape

(382688,)

In [141]:
#划分训练集、测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 0)

In [142]:
X_train.shape

(267881, 200)

In [143]:
train_x = X_train.T

In [144]:
train_x.shape

(200, 267881)

In [145]:
#将类别转换为one-hot表示，以用于训练
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
train_y0 = encoder.fit_transform(y_train)
train_y = train_y0.T

In [146]:
train_y.shape

(15, 267881)

In [147]:
train_y

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

#### NeuralNetwork

In [148]:
def initialize_parameters( n_x , n_h ,n_y):
    """
    参数：
        n_x - 输入层节点的数量
        n_h - 隐藏层节点的数量
        n_y - 输出层节点的数量
    
    返回：
        parameters - 包含参数的字典：
            W1 - 权重矩阵,维度为（n_h，n_x）
            b1 - 偏向量，维度为（n_h，1）
            W2 - 权重矩阵，维度为（n_y，n_h）
            b2 - 偏向量，维度为（n_y，1）

    """
    np.random.seed(2)
    W1 = np.random.randn(n_h,n_x) * 0.01 #(4,200)
    b1 = np.zeros(shape=(n_h, 1)) # (4,1)
    W2 = np.random.randn(n_y,n_h) * 0.01 #(15,4)
    b2 = np.zeros(shape=(n_y, 1)) # 15,1
    
    
    parameters = {"W1" : W1,
                  "b1" : b1,
                  "W2" : W2,
                  "b2" : b2 }
    
    return parameters

In [149]:
def sigmoid(x):
    s = 1/(1+np.exp(-x))
    return s

def forward_propagation( X , parameters ):
    """
    参数：
         X - 维度为（n_x，m）的输入数据。
         parameters - 初始化函数（initialize_parameters）的输出
    
    返回：
         A2 - 使用sigmoid()函数计算的第二次激活后的数值
         cache - 包含“Z1”，“A1”，“Z2”和“A2”的字典类型变量
     """
    W1 = parameters["W1"]#(4,200)
    b1 = parameters["b1"]#4,1)
    W2 = parameters["W2"]#(14,4)
    b2 = parameters["b2"]## 14,1
    
    #前向传播计算A2
    Z1 = np.dot(W1 , X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2 , A1) + b2
    A2 = sigmoid(Z2)
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return (A2, cache)

In [150]:
def compute_cost(A2,Y,parameters):
    """
    计算交叉熵成本，
    
    参数：
         A2 - 使用sigmoid()函数计算的第二次激活后的数值
         Y - "True"标签向量,维度为（1，数量）
         parameters - 一个包含W1，B1，W2和B2的字典类型的变量
    
    返回：
         成本 - 交叉熵成本
    """
    
    m = Y.shape[1]
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    
    #计算成本
    logprobs = logprobs = np.multiply(np.log(A2), Y) + np.multiply((1 - Y), np.log(1 - A2))
    cost = - np.sum(logprobs) / m
    cost = float(np.squeeze(cost))
    
    
    return cost

In [151]:
def backward_propagation(parameters,cache,X,Y):
    """ 
    参数：
     parameters - 包含我们的参数的一个字典类型的变量。
     cache - 包含“Z1”，“A1”，“Z2”和“A2”的字典类型的变量。
     X - 输入数据，维度为（2，数量）
     Y - “True”标签，维度为（1，数量）
    
    返回：
     grads - 包含W和b的导数一个字典类型的变量
    """
    m = X.shape[1]
    
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    
    A1 = cache["A1"]
    A2 = cache["A2"]
    
    dZ2= A2 - Y
    dW2 = (1 / m) * np.dot(dZ2, A1.T)
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.multiply(np.dot(W2.T, dZ2), 1 - np.power(A1, 2))
    dW1 = (1 / m) * np.dot(dZ1, X.T)
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2 }
    
    return grads

In [152]:
def update_parameters(parameters,grads,learning_rate=1.2):
    """  
    参数：
     parameters - 包含参数的字典类型的变量
     grads - 包含导数值的字典类型的变量
     learning_rate - 学习率
    
    返回：
     parameters - 包含更新参数的字典类型的变量
    """
    W1,W2 = parameters["W1"],parameters["W2"]
    b1,b2 = parameters["b1"],parameters["b2"]
    
    dW1,dW2 = grads["dW1"],grads["dW2"]
    db1,db2 = grads["db1"],grads["db2"]
    
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [153]:
def model(X,Y,n_h,num_iterations,print_cost=False):
    """
    参数：
        X - 数据集,维度为（200，示例数）
        Y - 标签，维度为（15，示例数）
        n_h - 隐藏层的数量
        num_iterations - 梯度下降循环中的迭代次数
        print_cost - 如果为True，则每1000次迭代打印一次成本数值
    
    返回：
        parameters - 模型学习的参数，它们可以用来进行预测。
     """
     
    np.random.seed(3) #指定随机种子
    n_x = layer_sizes(X, Y)[0] #
    n_y = layer_sizes(X, Y)[2] #
    
    parameters = initialize_parameters(n_x,n_h,n_y)
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    for i in range(num_iterations):
        A2 , cache = forward_propagation(X,parameters)
        cost = compute_cost(A2,Y,parameters)
        grads = backward_propagation(parameters,cache,X,Y)
        parameters = update_parameters(parameters,grads,learning_rate = 0.5)
        
        if print_cost:
            if i%1000 == 0:
                print("第 ",i," 次循环，成本为："+str(cost))
    return parameters

In [154]:
#预测函数
def predict(parameters,X):
    """
    使用学习的参数，为X中的每个示例预测一个类
    
    参数：
        parameters - 包含参数的字典类型的变量。
        X - 输入数据（n_x，m）
    
    返回
        predictions 
     
     """
    A2 , cache = forward_propagation(X,parameters) 
    predictions = A2
    return predictions

In [155]:
parameters = model(train_x, train_y, n_h = 5, num_iterations=10000, print_cost=True)

第  0  次循环，成本为：10.398739844009903
第  1000  次循环，成本为：2.0215408169583187
第  2000  次循环，成本为：1.726900300346474
第  3000  次循环，成本为：1.7193053938372131
第  4000  次循环，成本为：1.7152320981710192
第  5000  次循环，成本为：1.7234599264776855
第  6000  次循环，成本为：1.726509435379465
第  7000  次循环，成本为：1.68175537336197
第  8000  次循环，成本为：1.6663305333459606
第  9000  次循环，成本为：1.666206061728937


In [156]:
#预测和评估
def pre_score(parameters, X_test, y_test):
    
    #前向传播计算预测值
    test_x = X_test.T
    predictions = predict(parameters, test_x )
    pred = predictions.T
    preMax = np.argmax(pred, axis=1)
    
    #评估结果
    print(accuracy_score(preMax,y_test))
    print(classification_report(y_test, preMax,target_names=id_df['label'].values))
    return

pre_score(parameters, X_test ,y_test)

0.724302525107354
                    precision    recall  f1-score   support

      news_culture       0.66      0.71      0.69      8365
news_entertainment       0.69      0.80      0.74     11850
       news_sports       0.87      0.81      0.84     11155
      news_finance       0.68      0.56      0.61      8314
        news_house       0.78      0.75      0.77      5333
          news_car       0.79      0.82      0.81     10771
          news_edu       0.74      0.78      0.76      8154
         news_tech       0.72      0.76      0.74     12542
     news_military       0.68      0.63      0.65      7411
       news_travel       0.72      0.64      0.68      6340
        news_world       0.63      0.64      0.64      7987
  news_agriculture       0.62      0.75      0.68      5762
         news_game       0.78      0.77      0.78      8900
             stock       0.00      0.00      0.00       111
        news_story       0.60      0.18      0.27      1812

          accuracy  